In [18]:
import numpy as np
from scipy.sparse import csr_matrix
import pyriemann

In [19]:
load_mat = np.load("./adj_mat.npy")

In [20]:

def sgwt_raw_laplacian(B):
    B         = B.T;
    N         = B.shape[0] 
    degrees   = B.sum(axis=1)
    diagw     = np.diag(B)

    nj2,ni2   = B.nonzero() 
    w2        = np.extract(B!=0,B)
    ndind     = (ni2!=nj2).nonzero()
    ni        = ni2[ndind]
    nj        = nj2[ndind]
    w         = w2[ndind]
    di        = np.arange(0,N)
    #dL        = 1 - diagw / degrees       
    #dL[degrees==0] = 0
    #ndL       = -w / (np.sqrt(degrees[ni]*degrees[ni])).flatten() 
    L         = csr_matrix((np.hstack((-w,degrees-diagw)), (np.hstack((ni,di)), np.hstack((nj,di)))), shape=(N, N)).toarray()

    return L

In [21]:
N_layers = 2

Ls = []

for i in range(N_layers):
    Ls.append(sgwt_raw_laplacian(load_mat[:,:,i]))


In [22]:
L = np.stack(Ls,axis = 2)

In [25]:
def get_gm(L, S,n):
    L_reg = np.zeros((S, n, n))

    for i in range(S):
        L_reg[i,:,:] = L[:,:,i] +  0.001 * np.eye(n)
    
    
    L_geometric_mean = pyriemann.utils.mean.mean_riemann(L_reg)
    
    return L_geometric_mean

In [26]:
geometric_mean = get_gm(L, 2, load_mat.shape[0])

/home/keshav/miniconda3/envs/graphEnv/lib/python3.6/site-packages/pyriemann/utils/base.py:12: RuntimeWarning: invalid value encountered in sqrt
  eigvals = numpy.diag(operator(eigvals))
/home/keshav/miniconda3/envs/graphEnv/lib/python3.6/site-packages/pyriemann/utils/base.py:78: RuntimeWarning: invalid value encountered in sqrt
  isqrt = lambda x: 1. / numpy.sqrt(x)


ValueError: Covariance matrices must be positive definite. Add regularization to avoid this error.